In [11]:
# goal of this notebook is to find the best WR, RB opportunities 
# and back test last year's results to see if it works

In [12]:
# Dependencies
import pandas as pd
import numpy as np

In [20]:
# what csvs are available to use
!cd csvs; ls | grep .csv

defense-rankings.csv
offense-rankings.csv
offensive-line-rankings.csv
teams_locations_acronyms.csv


In [25]:
#!cd ./csvs/schedule-scrape/; ls

# where are CSVs
teams_csv = "./csvs/teams_locations_acronyms.csv"
current_schedule_csv = "./csvs/schedule-scrape/new-cleaned-schedule.csv"
defense_rankings_csv = "./csvs/defense-rankings.csv"
offense_rankings_csv = "./csvs/offense-rankings.csv"
ol_rankings_csv = "./csvs/offensive-line-rankings.csv"


# not using now
old_schedule_csv = "./csvs/schedule-scrape/old-cleaned-schedule.csv"

In [19]:
teams_df = pd.read_csv(teams_csv, 
                       encoding="iso-8859-1")
teams_df.head()

,acronym,location + team,location,team
0,ARI,Arizona Cardinals,Arizona,Cardinals
1,ATL,Atlanta Falcons,Atlanta,Falcons
2,BAL,Baltimore Ravens,Baltimore,Ravens
3,BUF,Buffalo Bills,Buffalo,Bills
4,CAR,Carolina Panthers,Carolina,Panthers


In [26]:
current_schedule_df = pd.read_csv(current_schedule_csv, 
                                  encoding="iso-8859-1", low_memory="False")
current_schedule_df.head()

,week,time,away,home,year
0,1,8:30 PM,Chiefs,Patriots,2017
1,1,1:00 PM,Jets,Bills,2017
2,1,1:00 PM,Eagles,Redskins,2017
3,1,1:00 PM,Raiders,Titans,2017
4,1,1:00 PM,Buccaneers,Dolphins,2017


In [ ]:
defense_rankings_df = pd.read_csv(defense_rankings_csv, 
                                  sep='\t', 
                                  encoding="iso-8859-1", 
                                  low_memory="False").rename(
    columns={'team': 'acronym', 'ranking': 'defense_ranking'})
defense_rankings_df.head()

In [ ]:
offense_rankings_df = pd.read_csv(offense_rankings_csv, sep='\t', 
                                  encoding="iso-8859-1", low_memory="False")
offense_rankings_df.head()

In [ ]:
ol_rankings_df = pd.read_csv(ol_rankings_csv, sep='\t', 
                             encoding="iso-8859-1", low_memory="False")
ol_rankings_df.head()

In [27]:
cur_sched_w_acr = pd.merge(teams_df[['acronym','team']], 
                           current_schedule_df[['week', 'away', 'home']], 
                           how='left', left_on=['team'], right_on=['away']).rename(
    columns={'acronym': 'away_acronym'})[['away_acronym', 'week', 
                                          'away', 'home']]

cur_sched_w_acr.head()

,away_acronym,week,away,home
0,ARI,1,Cardinals,Lions
1,ARI,2,Cardinals,Colts
2,ARI,5,Cardinals,Eagles
3,ARI,7,Cardinals,Rams
4,ARI,9,Cardinals,49ers


In [28]:
# have to merge again to itself to get the home team's acronym
cur_sched_w_acr = cur_sched_w_acr.merge(teams_df[['acronym','team']], 
                      how='left', 
                      left_on=['home'], 
                      right_on=['team']).rename(columns={
    'acronym': 'home_acronym'})[['week', 'away', 'away_acronym','home_acronym','home']]

cur_sched_w_acr.head()

,week,away,away_acronym,home_acronym,home
0,1,Cardinals,ARI,DET,Lions
1,2,Cardinals,ARI,IND,Colts
2,5,Cardinals,ARI,PHI,Eagles
3,7,Cardinals,ARI,STL,Rams
4,7,Cardinals,ARI,LAR,Rams


In [48]:
cur_sched_w_acr_wdef = cur_sched_w_acr.merge(defense_rankings_df[['acronym', 
                                                                  'defense_ranking']], 
                                             how='left', 
                                             left_on=['away_acronym'], 
                                             right_on=['acronym']).rename(columns={
    'defense_ranking': 'away_defense_ranking'})

cur_sched_w_acr_wdef[['week', 'away_acronym', 'away', 'home', 'away_defense_ranking']].head()

,week,away_acronym,away,home,away_defense_ranking
0,1,ARI,Cardinals,Lions,9.0
1,2,ARI,Cardinals,Colts,9.0
2,5,ARI,Cardinals,Eagles,9.0
3,7,ARI,Cardinals,Rams,9.0
4,9,ARI,Cardinals,49ers,9.0


In [14]:
# all of cardinals games
#cur_sched_w_acr[(cur_sched_w_acr['away'] == 'Cardinals') | (cur_sched_w_acr['home'] == 'Cardinals')]